# Selected Economic Characteristics: Poverty from the American Community Survey

**[Work in progress]**

This notebook downloads [selected economic characteristics (DP03)](https://data.census.gov/cedsci/table?tid=ACSDP5Y2018.DP03) from the American Community Survey 2018 5-Year Data.

Data source: [American Community Survey 5-Year Data 2018](https://www.census.gov/data/developers/data-sets/acs-5year.html)

Authors: Peter Rose (pwrose@ucsd.edu), Ilya Zaslavsky (zaslavsk@sdsc.edu)

In [1]:
import os
import pandas as pd
from pathlib import Path
import time

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
NEO4J_IMPORT = Path(os.getenv('NEO4J_IMPORT'))
print(NEO4J_IMPORT)

/Users/peter/Library/Application Support/com.Neo4j.Relate/data/dbmss/dbms-8bf637fc-0d20-4d9f-9c6f-f7e72e92a4da/import


## Download selected variables

* [Selected economic characteristics for US](https://data.census.gov/cedsci/table?tid=ACSDP5Y2018.DP03)

* [List of variables as HTML](https://api.census.gov/data/2018/acs/acs5/profile/groups/DP03.html) or [JSON](https://api.census.gov/data/2018/acs/acs5/profile/groups/DP03/)

* [Description of variables](https://www2.census.gov/programs-surveys/acs/tech_docs/subject_definitions/2018_ACSSubjectDefinitions.pdf)

* [Example URLs for API](https://api.census.gov/data/2018/acs/acs5/profile/examples.html)

### Specify variables from DP03 group and assign property names

Names must follow the [Neo4j property naming conventions](https://neo4j.com/docs/getting-started/current/graphdb-concepts/#graphdb-naming-rules-and-recommendations).

In [4]:
variables = {# PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL
             'DP03_0119PE': 'povertyAllFamiliesPct',
             'DP03_0128PE': 'povertyAllPeoplePct'
            }

In [5]:
fields = ",".join(variables.keys())

In [6]:
for v in variables.values():
    if 'Pct' in v:
        print('p.' + v + ' = toFloat(row.' + v + '),')
    else:
        print('p.' + v + ' = toInteger(row.' + v + '),')

p.povertyAllFamiliesPct = toFloat(row.povertyAllFamiliesPct),
p.povertyAllPeoplePct = toFloat(row.povertyAllPeoplePct),


In [7]:
print(len(variables.keys()))

2


## Download county-level data using US Census API

In [8]:
url_county = f'https://api.census.gov/data/2018/acs/acs5/profile?get={fields}&for=county:*'

In [9]:
df = pd.read_json(url_county, dtype='str')
df.fillna('', inplace=True)
df.head()

,0,1,2,3
0,DP03_0119PE,DP03_0128PE,state,county
1,6.9,8.2,02,195
2,39.0,40.5,02,158
3,10.6,14.7,04,013
4,16.3,21.3,04,009


##### Add column names

In [10]:
df = df[1:].copy() # skip first row of labels
columns = list(variables.values())
columns.append('stateFips')
columns.append('countyFips')
df.columns = columns

Remove Puerto Rico (stateFips = 72) to limit data to US States

TODO handle data for Puerto Rico (GeoNames represents Puerto Rico as a country)

In [11]:
df.query("stateFips != '72'", inplace=True)

Save list of state fips (required later to get tract data by state)

In [12]:
stateFips = list(df['stateFips'].unique())
stateFips.sort()
print(stateFips)

['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56']


In [13]:
df.head()

,povertyAllFamiliesPct,povertyAllPeoplePct,stateFips,countyFips
1,6.9,8.2,02,195
2,39.0,40.5,02,158
3,10.6,14.7,04,013
4,16.3,21.3,04,009
5,19.9,24.0,04,023


In [14]:
# Example data
df[(df['stateFips'] == '06') & (df['countyFips'] == '073')]

,povertyAllFamiliesPct,povertyAllPeoplePct,stateFips,countyFips
897,8.7,12.5,06,073


In [15]:
df['source'] = 'American Community Survey 5 year'
df['aggregationLevel'] = 'Admin2'

### Save data

In [16]:
df.to_csv(NEO4J_IMPORT / "03a-USCensusDP03PovertyAdmin2.csv", index=False)

## Download zip-level data using US Census API

In [17]:
url_zip = f'https://api.census.gov/data/2018/acs/acs5/profile?get={fields}&for=zip%20code%20tabulation%20area:*'

In [18]:
df = pd.read_json(url_zip, dtype='str')
df.fillna('', inplace=True)
df.head()

,0,1,2,3
0,DP03_0119PE,DP03_0128PE,state,zip code tabulation area
1,6.0,14.6,51,23833
2,21.2,19.6,51,23850
3,7.6,12.6,51,23851
4,9.1,11.8,51,23890


##### Add column names

In [19]:
df = df[1:].copy() # skip first row
columns = list(variables.values())
columns.append('stateFips')
columns.append('postalCode')
df.columns = columns

In [20]:
df.head()

,povertyAllFamiliesPct,povertyAllPeoplePct,stateFips,postalCode
1,6.0,14.6,51,23833
2,21.2,19.6,51,23850
3,7.6,12.6,51,23851
4,9.1,11.8,51,23890
5,30.1,61.2,51,23302


In [21]:
# Example data
df.query("postalCode == '90210'")

,povertyAllFamiliesPct,povertyAllPeoplePct,stateFips,postalCode
24526,5.8,7.7,06,90210


In [22]:
df['source'] = 'American Community Survey 5 year'
df['aggregationLevel'] = 'PostalCode'

### Save data

In [23]:
df.to_csv(NEO4J_IMPORT / "03a-USCensusDP03PovertyZip.csv", index=False)

## Download tract-level data using US Census API
Tract-level data are only available by state, so we need to loop over all states.

In [24]:
def get_tract_data(state):
    url_tract = f'https://api.census.gov/data/2018/acs/acs5/profile?get={fields}&for=tract:*&in=state:{state}'
    df = pd.read_json(url_tract, dtype='str')
    time.sleep(1)
    # skip first row of labels
    df = df[1:].copy()
    # Add column names
    columns = list(variables.values())
    columns.append('stateFips')
    columns.append('countyFips')
    columns.append('tract')
    df.columns = columns
    return df

In [25]:
df = pd.concat((get_tract_data(state) for state in stateFips))
df.fillna('', inplace=True)

In [26]:
df['tract'] = df['stateFips'] + df['countyFips'] + df['tract']

In [27]:
df['source'] = 'American Community Survey 5 year'
df['aggregationLevel'] = 'Tract'

In [28]:
# Example data for San Diego County
df[(df['stateFips'] == '06') & (df['countyFips'] == '073')].head()

,povertyAllFamiliesPct,povertyAllPeoplePct,stateFips,countyFips,tract,source,aggregationLevel
6,2.5,41.3,06,073,06073008339,American Community Survey 5 year,Tract
7,1.7,3.0,06,073,06073008347,American Community Survey 5 year,Tract
8,1.3,3.3,06,073,06073008354,American Community Survey 5 year,Tract
9,9.1,15.9,06,073,06073008505,American Community Survey 5 year,Tract
10,6.8,10.5,06,073,06073017604,American Community Survey 5 year,Tract


### Save data

In [29]:
df.to_csv(NEO4J_IMPORT / "03a-USCensusDP03PovertyTract.csv", index=False)

In [30]:
df.shape

(73056, 7)